In [1]:
] activate .

In [2]:
### Initialization

using Suppressor;

@suppress begin
    using PowerSystems;
end

using Logging
global_logger(Logging.SimpleLogger(global_logger().stream, Logging.Debug));
global_logger()



SimpleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x00000032) open, 0 bytes waiting))), Debug, Dict{Any,Int64}())

# Extending PowerSystems types

In this notebook, we will look at how to extend `PowerSystems.Component` types. 

Let's first read some data in:

In [3]:
base_dir = joinpath(dirname(dirname(pathof(PowerSystems))),"data")
include(joinpath(base_dir, "data_5bus.jl"));

We now have a `generators5` array.

In [4]:
generators5

7-element Array{Generator,1}:
 ThermalDispatch(name="Alta")         
 ThermalDispatch(name="Park City")    
 ThermalDispatch(name="Solitude")     
 ThermalDispatch(name="Sundance")     
 ThermalDispatch(name="Brighton")     
 RenewableFix(name="SolarBusC")       
 RenewableCurtailment(name="WindBusA")

Let's say we want to add a new `ThermalDispatch` type with `x`, `y` coordinates.

First we can create a coordinate struct ( The `convert` function is a helper function that you can ignore ).

In [5]:
struct Coordinate
    x::Float64
    y::Float64
end
Base.convert(::Type{Coordinate}, t::Tuple{T, U}) where T<:Number where U<:Number = Coordinate(t[1], t[2])

In order to extend a `ThermalDispatch` struct, we want to create a new struct that is the `subtype` of the `supertype` of `ThermalDispatch`. So let's find the supertype of `ThermalDispatch` first.

In [6]:
supertype(ThermalDispatch)

ThermalGen

We can `subtype` this in two ways:

```julia
struct MyCustomThermalDispatch1 <: ThermalGen
    location::Coordinate
    name::String
    available::Bool
    bus::Bus
    tech::Union{TechThermal,Nothing}
    econ::Union{EconThermal,Nothing}
end
```

*OR*

```julia
struct MyCustomThermalDispatch1 <: ThermalGen
    location::Coordinate
    generator::ThermalDispatch    
end
```

The first is straightforward to understand, easy to read and easy to write. However, it is not very friendly for long term stability and maintenaince.

This is because if any change is made to `PowerSystems.ThermalDispatch`'s internal layout, i.e. add a new field, rename a field, remove a field, change the type of a field etc, then we have to make the same change in our code. This can get cumbersome for large codebases or code evolving over time.

We'll use the second approach in this presentation. This allows us to use the same implementation details as `ThermalDispatch`, allowing us to reuse more code than option 1.

*Note*: Both above approaches are valid options depending on the goals of the project, and both have own their benefits and tradeoffs.

In [7]:
struct MyCustomThermalDispatch1 <: ThermalGen
    location::Coordinate
    generator::ThermalDispatch
end

The above technique is also commonly known as composition. However, this comes with a challenge.

Let's say that there is a function defined on the supertype of `ThermalDispatch` that uses the `.` operator. For example:

```julia
get_name(gen::Generator) = gen.name
```

This would work fine if we called it with `ThermalDispatch`.

```julia
get_name(gen::ThermalDispatch) # "Brighton"
```

However, if we call it with `MyCustomThermalDispatch1`, it will fail.

```julia
get_name(gen::MyCustomThermalDispatch1) # ERROR: type MyCustomThermalDispatch1 has no field name
```

If an instance of `MyCustomThermalDispatch1` is used by a caller of this function, it will result in a runtime error.
This is because we have changed the struct layout of the type that we extended.

We have to work around this of course, since we want our `MyCustomThermalDispatch1` to be used interchangably with `ThermalDispatch`. To do this, we need to implement all the functions for `MyCustomThermalDispatch1` that were implemented for `ThermalDispatch`.


We can find out all the functions defined that take a `ThermalDispatch` as an input argument:

In [8]:
methodswith(ThermalDispatch, supertypes=true)

[1] validate(generator::Generator) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/generator.jl:21
[2] show(io::IO, ::MIME{Symbol("text/plain")}, pst::Component) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/utils/print.jl:36
[3] show(io::IO, pst::Component) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/utils/print.jl:33

There are three methods here, that are essentially two functions:

`validate`, `show`

We can create an instance of `MyCustomThermalDispatch1` using the following:

In [9]:
gen = MyCustomThermalDispatch1(
    (0,0), 
    ThermalDispatch(
        "Brighton", true, nodes5[5],
        TechThermal(600.0, (min=0.0, max=600.0), 150.0, (min =-450.0, max=450.0), nothing, nothing),
        EconThermal(600.0, [(0.0, 0.0), (8.0,450.0), (10.0,600.0)], 0.0, 0.0, 0.0, nothing)
    )
)

MyCustomThermalDispatch1:
   location: Coordinate(0.0, 0.0)
   generator: ThermalDispatch(name="Brighton")

Let's see what happens when we call the `validate` function.

In [10]:
validate(gen)

ErrorException: type MyCustomThermalDispatch1 has no field name

One solution to this is to implement the `validate` function for our custom type and call the same function but pass it the instance of `ThermalDispatch`. This is also known as forwarding.

In [11]:
PowerSystems.validate(g::MyCustomThermalDispatch1) = PowerSystems.validate(g.generator)

In [12]:
validate(gen)

┌ Debug: Generator validation
│   generator.name = Brighton
│   is_valid = true
└ @ PowerSystems /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/generator.jl:23


true

In this case there's only a couple of functions implemented on `ThermalDispatch`. You can imagine another type where there are a larger number of functions implemented.

Additionally, we have only loaded `PowerSystems.jl`. There may be `N` number of packages that depend on `PowerSystems.jl` that implement different functions on a type provided by `PowerSystems`.

One option is to manually write out every function that is implemented and forward the functions appropriately. This allows for a great deal of manual control, but can be verbose and may involve a lot of code repetition. This additionally may not be maintainable.

The second option is to let Julia forward functions automatically for you. Here we use a package called `ReusePatterns`.

In [13]:
@suppress begin
    using ReusePatterns
end

Let's say we define the type as follows:

In [14]:
struct MyCustomThermalDispatch2 <: ThermalGen
    location::Coordinate
    generator::ThermalDispatch
end

We can `forward` all functions that we defined on `ThermalDispatch` when called on `MyCustomThermalDispatch2` 
to the appropriate attribute

In [15]:
@forward( (MyCustomThermalDispatch2, :generator), ThermalDispatch)

4 method(s) forwarded


In [16]:
gen = MyCustomThermalDispatch2((0,0), ThermalDispatch("Brighton", true, nodes5[5],
    TechThermal(600.0, (min=0.0, max=600.0), 150.0, (min =-450.0, max=450.0), nothing, nothing),
    EconThermal(600.0, [(0.0, 0.0), (8.0,450.0), (10.0,600.0)], 0.0, 0.0, 0.0, nothing)
))


ThermalDispatch:
   name: Brighton
   available: true
   bus: Bus(name="nodeE")
   tech: TechThermal
   econ: EconThermal

In [17]:
validate(gen)

┌ Debug: Generator validation
│   generator.name = Brighton
│   is_valid = true
└ @ PowerSystems /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/generator.jl:23


true

Note here that even the `show` function was forwarded. Any function found by the `@forward` macro will be forwarded.

This all or nothing behaviour may be what you desire, but there is also an option for controlled forwarding if you choose to do so.

In [18]:
@suppress begin
    using ReusePatterns
end

In [19]:
struct MyCustomThermalDispatch3 <: ThermalGen
    location::Coordinate
    generator::ThermalDispatch
end

Here we can pass the `validate` method array to the `@forward` macro.

In [20]:
methods_array = methods(validate).ms

[1] validate(p1::MyCustomThermalDispatch2) in PowerSystems at none:1
[2] validate(g::MyCustomThermalDispatch1) in Main at In[11]:1
[3] validate(sys::System) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/powersystem.jl:24
[4] validate(gen::GenClasses) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/generator.jl:4
[5] validate(bus::Bus) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/bus.jl:4
[6] validate(branch::Branch) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/branch.jl:4
[7] validate(generator::Generator) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/generator.jl:21
[8] validate(load::ElectricLoad) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/load.jl:5
[9] validate(storage::Storage) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/storage.jl:4

In [21]:
@forward( (MyCustomThermalDispatch3, :generator), ThermalDispatch, methods_array)

2 method(s) forwarded


In [22]:
gen = MyCustomThermalDispatch3((0,0), ThermalDispatch("Brighton", true, nodes5[5],
    TechThermal(600.0, (min=0.0, max=600.0), 150.0, (min =-450.0, max=450.0), nothing, nothing),
    EconThermal(600.0, [(0.0, 0.0), (8.0,450.0), (10.0,600.0)], 0.0, 0.0, 0.0, nothing)
))


MyCustomThermalDispatch3:
   location: Coordinate(0.0, 0.0)
   generator: ThermalDispatch(name="Brighton")

In [23]:
validate(gen)

┌ Debug: Generator validation
│   generator.name = Brighton
│   is_valid = true
└ @ PowerSystems /Users/$USER/.julia/packages/PowerSystems/a865r/src/validation/generator.jl:23


true

You can see that only `validate` was forwarded this time. 

## Optional: Custom Constructors

If you want to go the extra mile, you can define constructors that are implemented for `ThermalDispatch`. 

In [24]:
methods(ThermalDispatch)

# 3 methods for generic function "(::Type)":
[1] ThermalDispatch(; name, status, bus, tech, econ) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/models/generation/thermal_generation.jl:29
[2] ThermalDispatch(name::String, available::Bool, bus::Bus, tech::Union{Nothing, TechThermal}, econ::Union{Nothing, EconThermal}) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/models/generation/thermal_generation.jl:18
[3] ThermalDispatch(name, available, bus, tech, econ) in PowerSystems at /Users/$USER/.julia/packages/PowerSystems/a865r/src/models/generation/thermal_generation.jl:18

In [25]:
function MyCustomThermalDispatch1(; location=(0,0), kwargs...)
    generator = ThermalDispatch(kwargs...)
    MyCustomThermalDispatch(location, generator)
end

function MyCustomThermalDispatch1(args...)
    generator = ThermalDispatch(args...)
    location = (0, 0)
    MyCustomThermalDispatch1(location, generator)
end

MyCustomThermalDispatch1(
    name::String, 
    available::Bool, 
    bus::Bus, 
    tech::Union{Nothing, TechThermal}, 
    econ::Union{Nothing, EconThermal}
    ) = MyCustomThermalDispatch1((0,0), ThermalDispatch(name, available, bus, tech, econ))

MyCustomThermalDispatch1

And it is straighforward to add any extensions as necessary. Let's define one additional constructor here:

In [26]:
MyCustomThermalDispatch1(
    location::Union{Tuple{<:Number, <:Number}, Coordinate}, # Prefer not being as explicit in types unless required
    name::String, 
    available::Bool, 
    bus::Bus, 
    tech::Union{Nothing, TechThermal}, 
    econ::Union{Nothing, EconThermal}
    ) = MyCustomThermalDispatch1(location, ThermalDispatch(name, available, bus, tech, econ));

Now `MyCustomThermalDispatch1` can be constructed similarly to `ThermalDispatch `

In [27]:
MyCustomThermalDispatch1("Brighton", true, nodes5[5],
    TechThermal(600.0, (min=0.0, max=600.0), 150.0, (min =-450.0, max=450.0), nothing, nothing),
    EconThermal(600.0, [(0.0, 0.0), (8.0,450.0), (10.0,600.0)], 0.0, 0.0, 0.0, nothing)
)


MyCustomThermalDispatch1:
   location: Coordinate(0.0, 0.0)
   generator: ThermalDispatch(name="Brighton")